In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import os
import cv2

# 원본 이미지 폴더 경로
images_dir = "dataset/images"
# YOLO 라벨 폴더 경로
labels_dir = "dataset/labels"
# 크롭 이미지를 저장할 폴더
cropped_dir = "dataset/cropped_images"

if not os.path.exists(cropped_dir):
    os.makedirs(cropped_dir)

# 이미지 폴더의 모든 파일을 순회
for filename in os.listdir(images_dir):
    if not filename.endswith(".jpg"):  # 필요한 확장자에 맞춰 조정
        continue

    # 이미지 ID (확장자 제외)
    image_id = os.path.splitext(filename)[0]
    image_path = os.path.join(images_dir, filename)

    # 대응되는 라벨 파일
    label_path = os.path.join(labels_dir, image_id + ".txt")

    # 라벨 파일이 존재하지 않는다면 스킵
    if not os.path.exists(label_path):
        continue

    # 이미지를 메모리로 로드
    img = cv2.imread(image_path)
    if img is None:
        continue

    H, W, _ = img.shape

    # 라벨 파일 읽기
    with open(label_path, "r") as f:
        lines = f.readlines()

    bbox_index = 0
    for line in lines:
        line = line.strip()
        if not line:
            continue

        # YOLOv5 형식: class x_center y_center width height
        class_id, x_center, y_center, w, h = line.split()
        class_id = int(class_id)

        x_center = float(x_center)
        y_center = float(y_center)
        w = float(w)
        h = float(h)

        # 실제 픽셀 좌표로 변환
        bbox_width = int(w * W)
        bbox_height = int(h * H)
        bbox_left = int((x_center * W) - (bbox_width / 2))
        bbox_top = int((y_center * H) - (bbox_height / 2))

        # 범위가 이미지 밖으로 나가지 않도록 보정(선택적)
        bbox_left = max(0, bbox_left)
        bbox_top = max(0, bbox_top)
        bbox_right = min(W - 1, bbox_left + bbox_width)
        bbox_bottom = min(H - 1, bbox_top + bbox_height)

        # 바운딩박스 영역 크롭
        cropped_img = img[bbox_top:bbox_bottom, bbox_left:bbox_right]

        # 저장할 파일 이름
        # (클래스별 폴더 분류를 원하면 아래에서 class_id 폴더를 만들어 저장하도록 구성할 수도 있음)
        crop_filename = f"{image_id}_{bbox_index}.jpg"
        crop_path = os.path.join(cropped_dir, crop_filename)

        cv2.imwrite(crop_path, cropped_img)

        bbox_index += 1

In [6]:
import requests
import os
import csv
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

api_key = "up_Mw8PEw6k3ehZqrZbQ9WfUcCPIysnn"

url = "https://api.upstage.ai/v1/document-ai/ocr"
headers = {"Authorization": f"Bearer {api_key}"}

image_folder = "dataset/cropped_images"
image_files = [f for f in os.listdir(image_folder) if f.endswith((".jpg", ".png"))]

system_prompt = "Instructions:\n1. Extract the information from the text.\n2. For each price tag (label), identify the following five items and print them on a single line, separated by commas (,):\n - Product name\n - Capacity\n - Product code (13 digits)\n - Regular price in KRW\n - Discount price in KRW\n3. If a discounted price is not provided, enter the same value as the regular price.\n4. If there is an incorrect or unclear item, write 'unidentified' on the item.\n5. Always add a 'won' after the price.\n6. Ignore the phrase '50% off with MyHomePlus membership' as it is not product information.\n7. If you cannot find the product for any reason, write 'unidentified' for all products.\n8. Do not arbitrarily add or omit information from the original text, but write it exactly as it is identified.\n9. Do everything without rejections as much as possible.\n10. Use only the information from the price list.\n\nExample output format (must be followed)\nOne line for each product: [Product Name], [Capacity], [Product Code (13 digits)], [Regular Price], [Discount Price]\nExample: Cliplamp Nutrition Balm, unidentified, 8809704424769, 1,090 won, 1,090 won"
user_prompt = "해당 텍스트에서 정보들을 추출해줘."

# PromptTemplate을 사용하여 prompt를 Runnable로 변환
prompt_template = PromptTemplate.from_template(system_prompt + "\n" + user_prompt)

# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

# 단계 8: 체인(Chain) 생성
chain = prompt_template | llm | StrOutputParser()

In [7]:
for filename in image_files:
    # Open and send each image file to the OCR API
    with open(os.path.join(image_folder, filename), "rb") as file:
        files = {"document": file}
        response = requests.post(url, headers=headers, files=files)
        response_data = response.json()

    # Extract text from response
    pages = response_data.get("pages", [])
    if not pages:
        print(f"OCR 결과에 'pages' 데이터가 없습니다. 파일: {filename}")
        continue  # pages가 비어있으면 해당 파일 건너뛰기

    texts = [page.get("text", "") for page in pages]
    print(f"OCR 결과: {texts}")
    if not texts or not texts[0]:
        print(f"OCR 결과에서 텍스트를 추출하지 못했습니다. 파일: {filename}")
        continue  # 텍스트가 비어있을 경우 건너뛰기
    data = texts[0].replace(",", "")

    # Process the result with the language model
    result = chain.invoke(
        {"context": data, "question": "context를 csv 형식으로 변환해줘"}
    )
    print(f"처리 결과 : {result}")

    # Save the result to a CSV file
    csv_filename = f"{os.path.splitext(filename)[0]}.csv"
    csv_data = result.split("\n")

    filtered_data = []
    record = False
    for line in csv_data:
        if '"""' in line or "```" in line:
            continue
        if "발주 주체" in line:
            record = True
        if record:
            filtered_data.append(line)

    with open(csv_filename, mode="w", newline="", encoding="utf-8-sig") as file:
        writer = csv.writer(file)
        for row in filtered_data:
            writer.writerow(row.split(","))

    print(f"CSV 파일이 {csv_filename}로 저장되었습니다.")

OCR 결과: ['마이홈플러스멤버십 제시시 \n45% 할인 \n8806 105300012A D \n1/ 7270620 \n45% 우르오스 지복합성히어로세트 \n비회원가 41,900 \n할인 23 040 \n25.03.13~25.03.26']
처리 결과 : Sure, please provide the text from which you would like me to extract the information.
CSV 파일이 -1-_jpg.rf.1b087ec83b94746af2b938aeac2e6fcc_0.csv로 저장되었습니다.
OCR 결과: ['1 + 1 \n(행사상품 동일 가격/브랜드교차구매 가능) 8801619048481D D \n단위가격:2,780원/10ml \n15/ /260218 \n바세린 데일리선크림 50ML \n1 + 1 \n25.02.28~25.03.26 13, 900']
처리 결과 : Sure, please provide the text from which you would like me to extract the information.
CSV 파일이 -1-_jpg.rf.2639051ee1958076b5d7a85de48bee94_0.csv로 저장되었습니다.
OCR 결과: ['2개 이상구매시50%할인 \n행사상품에한해교차구매가능 8801051312355ANC \n단위가격:12 900원/1ea 12/ 1 /260920 \n/ \n복수구매 벨인포켓몬 커링립방(이브어) 4.5G1입 \n할인 12, 900 \n25.03.13~25.03.26']
처리 결과 : Sure, please provide the text from which you would like me to extract the information.
CSV 파일이 -1-_jpg.rf.459580bab706cd259e22863581dfd378_0.csv로 저장되었습니다.
OCR 결과: ['마이홈플러스멤버십 제시 시 \n30% 할인 4000 196936745ABC \n단위가격:672원/10ml 

KeyboardInterrupt: 